In [ ]:
import tensorflow as tf
import string
import requests
import pandas as pd

In [ ]:
response = requests.get('https://raw.githubusercontent.com/laxmimerit/poetry-data/master/adele.txt')

In [ ]:
data = response.text.splitlines()
print(data[:10]) # 10 sample

['Looking for some education', 'Made my way into the night', 'All that bullshit conversation', "Baby, can't you read the signs? I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes", "It's more than enough"]


In [ ]:
len(data)

2400

In [ ]:
# Total word number is:
len(" ".join(data))

91330

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(data) #it's going to fit on the data in the forms of lines.

In [ ]:
encoded_text=tokenizer.texts_to_sequences(data)
encoded_text[:10]
# These number actually comes from vocabulary
# it assigns number to each words

[[254, 21, 219, 725],
 [117, 8, 80, 153, 3, 133],
 [14, 10, 726, 727],
 [41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3, 518, 41],
 [1, 23, 107, 189, 300, 9, 57],
 [286, 35, 46, 10, 230],
 [2, 83, 134, 4, 519, 8, 120],
 [1, 37, 520, 102, 19, 27, 25, 254, 21, 328, 11],
 [27, 209, 11, 13, 9, 124],
 [42, 67, 210, 125]]

In [ ]:
wc=tokenizer.word_counts
# word frequency

In [ ]:
wi=tokenizer.word_index

In [ ]:
print(f"Number of unique words and total vocab size: {len(tokenizer.word_counts)+1}")
vocab_size=len(tokenizer.word_counts)+1

Number of unique words and total vocab size: 1396


In [ ]:
x=["play this song"]

In [ ]:
tokenizer.texts_to_sequences(x)

[[241, 44, 409]]

In [ ]:
 encoded_text[:10]

[[254, 21, 219, 725],
 [117, 8, 80, 153, 3, 133],
 [14, 10, 726, 727],
 [41, 56, 2, 603, 3, 728, 1, 68, 517, 2, 40, 3, 518, 41],
 [1, 23, 107, 189, 300, 9, 57],
 [286, 35, 46, 10, 230],
 [2, 83, 134, 4, 519, 8, 120],
 [1, 37, 520, 102, 19, 27, 25, 254, 21, 328, 11],
 [27, 209, 11, 13, 9, 124],
 [42, 67, 210, 125]]

In [ ]:
data_list=[]
for i in encoded_text:
    if len(i)>1:
        for j in range(2,len(i)):
            data_list.append(i[:j])
#             print(i[:j]) # if you want to check data

In [ ]:
max_length=20
#max length of line is 20 token per line in our poetry

In [ ]:
sequences=pad_sequences(data_list,maxlen=max_length,padding="pre") # we set the lenght size equal to 20
sequences[:5] # 5 sample

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0, 254,  21],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0, 254,  21, 219],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0, 117,   8],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0, 117,   8,  80],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0, 117,   8,  80, 153]], dtype=int32)

In [ ]:
sequences.shape

(14231, 20)

In [ ]:
X=sequences[:,:-1]
y=sequences[:,-1]
print("X values")
print(X[:5]) # 5 sample
print("-"*30)
print("X values")
print(y[:5]) # 5 sample

X values
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  254]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 254
   21]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  117]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 117
    8]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 117   8
   80]]
------------------------------
X values
[ 21 219   8  80 153]


In [ ]:
X.shape,y.shape

((14231, 19), (14231,))

In [ ]:
y=to_categorical(y,num_classes=vocab_size)
print(y[:5]) # 5 sample
print("Shape of y",y.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Shape of y (14231, 1396)


In [ ]:
print("Shape of X",X.shape)
seq_length=X.shape[1]
seq_length

Shape of X (14231, 19)


19

#### Building a LSTM Model

In [ ]:
model=Sequential()
model.add(Embedding(vocab_size,50,input_length=seq_length))
#The first layer is the Embedded layer that uses 50-length vectors
#return_sequences=True because we add another LSTM
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation="relu"))
model.add(Dense(vocab_size,activation="softmax")) # we use softmax because there is multiclasses

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
model.fit(X,y,batch_size=32,epochs=150)

Epoch 1/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 24s 45ms/step - accuracy: 0.0390 - loss: 5.9798
Epoch 2/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 21s 47ms/step - accuracy: 0.0472 - loss: 5.2928
Epoch 3/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 40s 44ms/step - accuracy: 0.0552 - loss: 5.0594
Epoch 4/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 21s 47ms/step - accuracy: 0.0711 - loss: 4.8831
Epoch 5/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 40s 45ms/step - accuracy: 0.0779 - loss: 4.7761
Epoch 6/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 20s 44ms/step - accuracy: 0.0959 - loss: 4.6288
Epoch 7/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 19s 43ms/step - accuracy: 0.1048 - loss: 4.5186
Epoch 8/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 22s 47ms/step - accuracy: 0.1152 - loss: 4.3792
Epoch 9/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 39s 43ms/step - accuracy: 0.1345 - loss: 4.2408
Epoch 10/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 22s 46ms/step - accuracy: 0.1519 - loss: 4.0811
Epoch 11/150
445/445 ━━━━━━━━━━━━━━━━━━━━ 41s 47ms/step - accuracy: 0.1686 - loss: 3.9538
Epoch 12/150
445/44

KeyboardInterrupt: 

### Text Generation

In [ ]:
text_lenght= 15
def generate_text(input_text, no_lines):
    general_text=[]
    for i in range(no_lines):
        text=[]
        for _ in range(text_lenght):
            encoded=tokenizer.texts_to_sequences([input_text])
            encoded=pad_sequences(encoded,maxlen=seq_length,padding="pre")
            y_pred=np.argmax(model.predict(encoded),axis=-1) # it will generate a word index, loop up into dictionary containing word index
            predicted_word=""
            for word,index in tokenizer.word_index.items():
                if index==y_pred:
                    predicted_word=word
                    break
            input_text=input_text +' '+ predicted_word
            text.append(predicted_word)
        input_text=text[-1]
        text=" ".join(text) # input text will be the last word of first created line
        general_text.append(text)
    return general_text

In [ ]:
import pickle
# Load the tokenizer
with open('tokenizer1.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)
# Assume `model` is your trained model
model.save('model1.keras', save_format='keras')

In [ ]:
input_text="me"
text_produced=generate_text(input_text,6)
text_produced

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 528ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━

['in the absence of dawn and the forests of the earth for fire of the',
 'scars of your love they leave me you were falling of down so you are',
 'the wonders of this have goodbye how the signs i make my heart i to',
 'make you feel my love the storms are raging on the rolling for it all',
 "the things you'd say they were never true never know we ain't kids no waste",
 "her you can did to whenever you make it want me to stop i'll all"]

In [ ]:
input_text="i want to see you"
text_produced=generate_text(input_text,6)
text_produced

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━

["told me i'm giving you you so know i could want you all i say",
 "it ain't so say it ain't so say it ain't so say it ain't and",
 "i can tell that he'll be there for life with your loving there ain't no",
 'one has me like you and the seasons of every would too to and your',
 'name i set fire to the river but let the sky fall to find of',
 'when we were young let me photograph you in this stay in the air i']

In [ ]:
input_text="i want to see you"
text_produced=generate_text(input_text,6)
text_produced

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━

["told me i'm giving you you so know i could want you all i say",
 "it ain't so say it ain't so say it ain't so say it ain't and",
 "i can tell that he'll be there for life with your loving there ain't no",
 'one has me like you and the seasons of every would too to and your',
 'name i set fire to the river but let the sky fall to find of',
 'when we were young let me photograph you in this stay in the air i']